In [1]:
from astropy.io import fits
from astropy.time import Time
import numpy as np
from pyvo.dal import tap

In [2]:
# For scans - 46575 46576 46577 46578 46579 46580 46585 46586 46587 46588 46591
prov_ids = [
            ["APEXBOL.2022-08-24T12:46:40.000", "APEX-46575-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T12:54:33.000", "APEX-46576-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:00:39.000", "APEX-46577-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:13:50.000", "APEX-46578-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:27:16.000", "APEX-46579-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:40:27.000", "APEX-46580-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:58:26.000", "APEX-46585-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:11:39.000", "APEX-46586-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:24:53.000", "APEX-46587-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:38:05.000", "APEX-46588-2022-08-24-E-0110.C-4194A-2022"],
            # ["APEXBOL.2022-08-24T14:50:21.000", "APEX-46590-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:58:53.000", "APEX-46591-2022-08-24-E-0110.C-4194A-2022"]
            ]

prov_ids = np.array(prov_ids)

In [3]:
input_dir = "./batch_30790/"
input_file = "Orion_CONCERTO_ESO_2all_cont.fits"

output_dir = "./batch_30931/"
output_file = "Orion_CONCERTO_ESO_2all_whitelight"

hdul = fits.open(f"{input_dir}/{input_file}")
hdul.info()

Filename: ./batch_30790//Orion_CONCERTO_ESO_2all_cont.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0                1 PrimaryHDU      49   ()      
  1  CONTMAP       1 ImageHDU        81   (360, 360)   float32   
  2  CONTNOISE     1 ImageHDU        82   (360, 360)   float32   
  3  CONTINT       1 ImageHDU        82   (360, 360)   float64   


In [4]:
header_0 = hdul[0].header
header_1 = hdul[1].header
header_2 = hdul[2].header

In [5]:
# Make TAP query to get observation details
def tap_query(query, ESO_TAP_OBS="https://archive.eso.org/tap_obs"):
    """Function to perform a TAP query to the ESO archive."""
    tapobs = tap.TAPService(ESO_TAP_OBS)
    result = tapobs.search(query=query, maxrec=1000).to_qtable()
    return result

prog_id = "0110.C-4194(A)"
object_name = "Orion"

query = f"""SELECT dp_id, exposure, prog_id, object, dp_tech, instrument, ra, dec, exp_start, origfile
            FROM dbo.raw
            WHERE dp_id like 'APEXBOL.%%'
                AND object = '{object_name}'
                AND prog_id = '{prog_id}'
                AND dp_cat = 'SCIENCE'""" 

result = tap_query(query)

# Mask out scan not used (i.e. only 11 out of 12 were used)
# 46590 not used 
mask = []
for origfile in result["origfile"]:
    if "46590" in origfile:
        mask.append(False)
    else:
        mask.append(True)
result = result[mask]

# Get the first and last exposure start times
exp_start = result["exp_start"]
exposure = result["exposure"]
dp_id = result["dp_id"]

# Get total exposure time 
texptime = float(np.sum(exposure).to("s").value)

mjdobs = exp_start[0]
mjdend = exp_start[-1]
dateobs = mjdobs.replace("Z", "")

mjdobs = Time(mjdobs, scale="utc")
mjdend = Time(mjdend, scale="utc")

mjdobs = float(round(mjdobs.mjd, 5))
mjdend = round(float(mjdend.mjd) + float(exposure[-1].to("d").value), 5)

result

dp_id,exposure,prog_id,object,dp_tech,instrument,ra,dec,exp_start,origfile
,s,,,,,deg,deg,,
object,float32,object,object,object,object,float64,float64,object,object
APEXBOL.2022-08-24T12:46:40.000,330.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T12:46:39.100Z,APEX-46575-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T12:54:33.000,360.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T12:54:32.100Z,APEX-46576-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:00:39.000,630.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:00:39Z,APEX-46577-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:13:50.000,720.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:13:50Z,APEX-46578-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:27:16.000,630.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:27:16Z,APEX-46579-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:40:27.000,720.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:40:26.100Z,APEX-46580-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:58:26.000,630.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:58:26Z,APEX-46585-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T14:11:39.000,720.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T14:11:38.100Z,APEX-46586-2022-08-24-E-0110.C-4194A-2022


In [6]:
del header_0["PSF_FWHM"]
del header_0["PHOTZP"]
del header_0["PHOTSYS"]
del header_0["ABMAGLIM"]
del header_0["ABMAGSAT"]
del header_0["COMMENT"]
del header_0["FILTER"]

header_0.set("PRODCATG", "ANCILLARY.IMAGE.WHITELIGHT", "Data product category", after="EXTEND")
header_0.set("BMAJ", header_0["SKY_RES"]/3600., "Major axis beam size (deg)")
header_0.set("BMIN", header_0["SKY_RES"]/3600., "Minor axis beam size (deg)")
header_0.set("BPA", 0.0, "Position angle of the beam (deg)")
header_0.set("INSTRUME", "APEXBOL", "Instrument type")
header_0.set("FEBE1", "CONCERTO-CONCERTOBE", "Frontend-Backend")
header_0.set("TIMESYS", "TAI", "Time system for MJD")
header_0.set("BNOISE", 0.0, "Median rms (Jy)")
header_0.set("FLUXERR", 0.15, "Flux error (15%)")
header_0.set("PROG_ID", prog_id, "Program ID")
header_0.set("MJD-OBS", mjdobs, "MJD of observation") # From raw file header... 
header_0.set("MJD-END", mjdend, "MJD of end of observation")
header_0.set("TEXPTIME", texptime, "Total exposure time (s)")
header_0.set("EXPTIME", texptime, "Total exposure time (s)")

header_0.set("DATE-OBS", dateobs, "Date of observation in ISO format")
date = Time.now().iso.replace(" ", "T")
header_0.set("DATE", date, "Date of file creation in ISO format")

header_0.set("OBSTECH", "SPECTRUM", "Technique of observation")

for i, prov_id in enumerate(prov_ids[:,0]):
    header_0.set(f"PROV{i+1}", prov_id, f"Provenance ID {i+1}")

header_0

SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                   32 /                                                
NAXIS   =                    0 /                                                
EXTEND  =                    T /Extensions may be present                       
PRODCATG= 'ANCILLARY.IMAGE.WHITELIGHT' / Data product category                  
ORIGIN  = 'APEX    '           /                                                
TELESCOP= 'APEX-12m'           /                                                
INSTRUME= 'APEXBOL '           / Instrument type                                
OBJECT  = 'Orion   '           /                                                
EQUINOX =              2000.00 /                                                
EXPTIME =               6810.0 / Total exposure time (s)                        
TEXPTIME=               6810.0 / Total exposure time (s)                        
MJD-OBS =           59815.53

In [7]:
hdul_cube = fits.open('./batch_30931/Orion_CONCERTO_ESO_2all_tot.fits')
wavelmin = hdul_cube[0].header["WAVELMIN"]
wavelmax = hdul_cube[0].header["WAVELMAX"]

header_0.set("WAVELMIN", wavelmin, "[nm] Minimum wavelength")
header_0.set("WAVELMAX", wavelmax, "[nm] Maximum wavelength")

In [8]:
for header in [header_1, header_2]:
    del header["FLUXERR"]
    del header["PHOTZP"]
    del header["PHOTSYS"]
    del header["ABMAGLIM"]
    del header["ABMAGSAT"]
    del header["SKY_RES"]
    del header["PSF_FWHM"]
    del header["SPEC_RES"]
    del header["WAVELMIN"]
    del header["WAVELMAX"]
    del header["PROCSOFT"]
    del header["FLUXCAL"]
    del header["TEXPTIME"]
    del header["EXPTIME"]
    del header["EQUINOX"]
    del header["INSTRUME"]
    del header["TELSCOP"]
    del header["ORIGIN"]
    del header["MJD-OBS"]
    del header["MJD-END"]
    del header["PROG_ID"]
    del header["REFERENC"]
    del header["DATE"]
    del header["DATE-OBS"]
    del header["NCOMBINE"]
    del header["PROV1"]
    del header["OBSTECH"]
    del header["MAPMODE"]
    del header["FEBE1"]
    del header["Comment"]
    del header["History"]
    del header["FILTER"]
    del header["EXTNAME"]
    if "EXTEND" in header:
        del header["EXTEND"]

In [9]:
del header_1["QUALDATA"] # No quality flags in data cube... 
del header_2["QUALDATA"] # No quality flags in data cube... 
del header_2["ERRDATA"] 

header_1.set("EXTNAME", "DATA_EXT")
header_1.set("ERRDATA", "STAT_EXT")

header_2.set("EXTNAME", "STAT_EXT")
header_2.set("HDUCLAS3", "RMSE")
header_2.set("SCIDATA", "DATA_EXT")

In [10]:
header_0

SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                   32 /                                                
NAXIS   =                    0 /                                                
EXTEND  =                    T /Extensions may be present                       
PRODCATG= 'ANCILLARY.IMAGE.WHITELIGHT' / Data product category                  
ORIGIN  = 'APEX    '           /                                                
TELESCOP= 'APEX-12m'           /                                                
INSTRUME= 'APEXBOL '           / Instrument type                                
OBJECT  = 'Orion   '           /                                                
EQUINOX =              2000.00 /                                                
EXPTIME =               6810.0 / Total exposure time (s)                        
TEXPTIME=               6810.0 / Total exposure time (s)                        
MJD-OBS =           59815.53

In [11]:
header_1

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  360 /                                                
NAXIS2  =                  360 /                                                
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =        180.000000000 / Pixel coordinate of reference point            
CRPIX2  =        180.000000000 / Pixel coordinate of reference point            
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TAN'        

In [12]:
header_2

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  360 /                                                
NAXIS2  =                  360 /                                                
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =        180.000000000 / Pixel coordinate of reference point            
CRPIX2  =        180.000000000 / Pixel coordinate of reference point            
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TAN'        

In [13]:
hdul_new = hdul.copy()

In [14]:
hdul_new.writeto(f"{output_dir}/{output_file}.fits", checksum=True, overwrite=True)